In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer

In [4]:
#creating a Sample Data Frame
dataframe = spark.createDataFrame([
     (0, "Using Spark Is Great"),
     (1, "We are learning Spark"),
     (2, "Spark is better than Hadoop")                              
],["id" , "sentence"])

In [5]:
dataframe.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Using Spark Is Great|
|  1|We are learning S...|
|  2|Spark is better t...|
+---+--------------------+



In [6]:
# Tokenize sentences
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer

Tokenizer_2085547920a2

In [7]:
# Transform and show DataFrame
tokenized_df = tokenizer.transform(dataframe)
tokenized_df.show(truncate=False)

+---+---------------------------+---------------------------------+
|id |sentence                   |words                            |
+---+---------------------------+---------------------------------+
|0  |Using Spark Is Great       |[using, spark, is, great]        |
|1  |We are learning Spark      |[we, are, learning, spark]       |
|2  |Spark is better than Hadoop|[spark, is, better, than, hadoop]|
+---+---------------------------+---------------------------------+



In [8]:
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)

In [9]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [10]:
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [11]:
# Create our Tokenizer
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

In [12]:
# Transform DataFrame
tokenized_df = tokenizer.transform(dataframe)

In [14]:
#Select the needed Columns and don't Truncate results
tokenized_df.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

+---+---------------------------+---------------------------------+------+
|id |sentence                   |words                            |tokens|
+---+---------------------------+---------------------------------+------+
|0  |Using Spark Is Great       |[using, spark, is, great]        |4     |
|1  |We are learning Spark      |[we, are, learning, spark]       |4     |
|2  |Spark is better than Hadoop|[spark, is, better, than, hadoop]|5     |
+---+---------------------------+---------------------------------+------+

